In [1]:
from othello_rules import *
from othello_net import *
from tensorflow.python.framework import ops
from datetime import datetime
from example_states import *
from feature_extractor import *
from training_utils import *
import numpy as np


In [88]:
def value_error(data_path, sess, diagnostics=False):
    errors = []
    validation_matches = get_all_matches(data_path)
    #XXX: Delete this line when testing is faster
    validation_matches = validation_matches[0:5]
    for i in range(len(validation_matches)):
        if diagnostics:
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>new game")
        match = validation_matches[i]
        raw_match_movelist = match[8:]
        unpacked_movelist = unpack('b'*60, raw_match_movelist)
        black_score = match[6]
        unpacked_black_score = unpack('b', black_score)
        winner = -1 if unpacked_black_score[0] > 32 else 1
        board = initialize_game()
        player = -1
        if diagnostics:
            print(unpacked_black_score)
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>winner is " + str(winner))
        for move in unpacked_movelist:
            if move == 0:
                break
            feature_path = 'cache/validation/features/features_' + str(i) + "_" + str(move) + ".npy"
            label_path = 'cache/validation/labels/labels_' + str(i) + "_" + str(move) + ".npy"
            #XXX: FIX THIS
            if os.path.isfile(feature_path) and os.path.isfile(label_path):
                features = np.load(feature_path)
            else:
                #XXX: FIX THIS
                #features = board_to_sanity(board, player)
                features = board_to_input(board, player)
                
            input_batch = [features]
            s = 1 if player * (-1) == winner else 0
            if s == 1:
                r = np.array([0, 1])
            else:
                r = np.array([1, 0])
        
            label_batch = [r]
            
            estimate = sess.run(score_out, feed_dict={img_data:input_batch, keep_prob:1.0})
            if diagnostics:
                print("-new state-")
                print("actual value: " + str(r))
                print("estimate: " + str(estimate))
            error = sess.run(loss, feed_dict={img_data:input_batch, ground_truths: label_batch, keep_prob:1.0})
            errors.append(error)
            board = make_move(board, move, player)
            if player is 1:
                player = -1
            else:
                player = 1
            legal_moves = find_legal_moves(board, player)
            if len(legal_moves) == 0:
                if player is 1:
                    player = -1
                else:
                    player = 1
        if diagnostics:
            print("<<<<<<<<<<<<<<<<<<<<<<actual winner is " + str(get_winner(board, -1, 1)))
    return np.sum(errors) / len(errors)
            
def create_sanity_net():
    with tf.name_scope("becoming_insane"):
        img_data = tf.placeholder(tf.float32, shape=[None, 8, 8, 6], name="img_data")
        keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        
        img_flat = tf.reshape(img_data, [-1, 8*8*6])
        
        
        W = tf.Variable(tf.truncated_normal([8*8*6, 1], stddev=0.1))
        b = tf.Variable(tf.constant(0.1, shape=[1]))
        
        value_out = tf.nn.relu(tf.matmul(img_flat, W) + b)
        prediction = tf.nn.sigmoid(value_out) 
        
        label = tf.placeholder(tf.float32, shape=[None,  1], name="ground_truths")
        learn_rate = tf.placeholder(tf.float32, name="eta")
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(value_out, label))
        
        policy_network_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        optimizer = tf.train.AdamOptimizer(learn_rate)
        train_step = optimizer.minimize(loss)
        sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
        return sess.graph, img_data, train_step, optimizer, label, loss, prediction, keep_prob, learn_rate, value_out
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
def conv_weights2(i, o):
    k = 2
    shape = [k, k, i, o]
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def create_value_net():
    with tf.name_scope("value_network"):
        img_data = tf.placeholder(tf.float32, shape=[None, 8, 8, 28], name="img_data")
        keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        #convolutional layers 
        i = 28; o = 32;
        conv1_weights = conv_weights(i, o)
        conv1_biases = conv_biases(o)
        conv1_out = conv_nonparams(img_data, conv1_weights, conv1_biases, keep_prob)
        pool1 = max_pool_2x2(conv1_out)
        i = o; o = 48;
        conv2_weights = conv_weights(i, o)
        conv2_biases = conv_biases(o)
        conv2_out = conv_nonparams(pool1, conv2_weights, conv2_biases, keep_prob)
        pool2 = max_pool_2x2(conv2_out)
        i = o; o = 64;
        conv3_weights = conv_weights2(i, o)
        conv3_biases = conv_biases(o)
        conv3_out = conv_nonparams(pool2, conv3_weights, conv3_biases, keep_prob)
        pool3 = max_pool_2x2(conv3_out)
        k = 1; i = o; o = 2;
        score_weights = weight_variable([k, k, i, o])
        score_biases = bias_variable([o])
        score_out = conv_nonparams(pool3, score_weights, score_biases, keep_prob)
        score_out_reshaped = tf.reshape(score_out, [-1, 1*1*2])
        prediction = softmax(score_out_reshaped, o)
        
        
        #W = tf.Variable(tf.truncated_normal([8*8*2, 2], stddev=1.0))
        #b = tf.Variable(tf.constant(-0.1, shape=[2]))
        # Construct model
        #value_out = tf.nn.relu(tf.matmul(score_out_reshaped, W) + b)
        #value_out_drop = tf.nn.dropout(value_out, keep_prob)
        #prediction = tf.nn.softmax(value_out_drop) # Softmax
        #training block:
        label = tf.placeholder(tf.float32, shape=[None,  2], name="ground_truths")
        learn_rate = tf.placeholder(tf.float32, name="eta")
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(score_out_reshaped, label))
        policy_network_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        #print(policy_network_variables)
        reg_loss = tf.reduce_sum([tf.reduce_sum(tf.square(x)) for x in policy_network_variables])
        optimizer = tf.train.AdamOptimizer(learn_rate)
        train_step = optimizer.minimize(loss)
        sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
        return sess.graph, img_data, train_step, optimizer, label, loss, prediction, keep_prob, learn_rate, score_out

    # MAKE SURE that the topology of this network is the same as the policy network!

In [ ]:
ops.reset_default_graph()
graph, img_data, train_step, optimizer, ground_truths, loss, pred_up, keep_prob, learn_rate, score_out = create_value_net()
saver = tf.train.Saver()
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
init_op = tf.initialize_all_variables()
sess.run(init_op)
current_model = "models/sl-v/tryagain24.ckpt"
if os.path.isfile(current_model):
    saver.restore(sess, current_model)
validation_path = "validation/"
matches = get_all_matches('training/')
lenmatches = len(matches)
print(lenmatches)

# Byrjum þjálfunina
print("start training")
#print("starting error:" + str(avg_error(validation_path, sess)))
#print('%s: Step %d: Prediction accuracy = %.2f' % (datetime.now(), 0,
#                                                      prediction_accuracy()/float(60)))
diagn = False
if diagn:
    iterations = 1
else:
    iterations = 300
prev_stop = 1300
probs = 1
start = 2e-5
for i in range(prev_stop, prev_stop+iterations):
    #TODO: Skrifa þetta fall
    #input_batch, label_batch = prepare_train_batch(train_ids, batch_size, do_flips, do_rots, data_path)
    current_match = matches[i]
    raw_match_movelist = current_match[8:]
    unpacked_movelist = unpack('b'*60, raw_match_movelist)
    black_score = current_match[6]
    unpacked_black_score = unpack('b', black_score)
    
    #XXX: Temporary check for the value network, do this properly later
    if unpacked_black_score[0] > 32:
        winner = -1
    else:
        winner = 1
    
    board = initialize_game()
    training_stability = np.zeros((8,8))
    player = -1
    
    # One training batch is all the data from one match
    input_batch = []
    label_batch = []
    
    for move in unpacked_movelist:
        if move == 0:
            break
        feature_path = 'cache/training/features/features_' + str(i) + "_" + str(move) + ".npy"
        label_path = 'cache/training/labels/labels_' + str(i) + "_" + str(move) + ".npy"
        #XXX: UNDO THIS
        if os.path.isfile(feature_path) and os.path.isfile(label_path):
            try:
                features = np.load(feature_path)
                label = np.load(label_path)
            except:
                print("data corruption in match " + str(i))
                
                #features = board_to_input(board, player)
                #XXX: UNDO THIS
                features = board_to_input(board, player)
                #features = board_to_sanity(board, player)
                label = prepare_data(move_to_label(move))
        else:
            #features = board_to_sanity(board, player)
            #XXX: UNDO THIS
            features = board_to_input(board, player)
            label = prepare_data(move_to_label(move))
        s = 1 if player * (-1) == winner else 0
        if s == 1:
            r = np.array([0, 1])
        else:
            r = np.array([1, 0])
        #print(r)
        if np.random.rand()>0.7:
            input_batch.append(features)
            label_batch.append(r)
            label_batch.append(r)
            label_batch.append(r)
            label_batch.append(r)
            # Now we add 3 reflections of the game state
            # which is done by flipping the board over one diagonal        
            move_upright = flip_move_upright(move)
            features_upright = flip_features(features, 'upright')
            input_batch.append(features_upright)
            # Then the other diagonal
            move_upleft = flip_move_upleft(move)
            features_upleft = flip_features(features, 'upleft')
            input_batch.append(features_upleft)
            # Then both diagonals
            move_both = flip_move_upright(flip_move_upleft(move))
            features_both = flip_features(features, 'both')
            input_batch.append(features_both)

        board = make_move(board, move, player)
        
        if player is 1:
            player = -1
        else:
            player = 1
        legal_moves = find_legal_moves(board, player)
        if len(legal_moves) == 0:
            if player is 1:
                player = -1
            else:
                player = 1
    eta = start
    #label_batch = [label_batch]
    _, scorez = sess.run([train_step, score_out],
                           feed_dict={img_data:input_batch,
                                            ground_truths: label_batch,
                                            keep_prob:probs,
                                            learn_rate:eta})
    #print("score out is")
    #print(scorez)

    if (i % 20 is 0) and (i > 0 and i < 8000) or (i+1) == (iterations+prev_stop) or (i % 1000 is 0):  
        print('%s, Step %d, Loss = %.6f' % (datetime.now().strftime("%d. %b %H:%M:%S"), i,
                                                                          value_error(validation_path, sess, diagn)))
        save_path = saver.save(sess, current_model)

print("done")

117298
start training
12. Oct 23:01:02, Step 1000, Loss = 0.693464
12. Oct 23:01:05, Step 1020, Loss = 0.694169
12. Oct 23:01:08, Step 1040, Loss = 0.693455
12. Oct 23:01:10, Step 1060, Loss = 0.699145
12. Oct 23:01:13, Step 1080, Loss = 0.699784
12. Oct 23:01:16, Step 1100, Loss = 0.700869
12. Oct 23:01:19, Step 1120, Loss = 0.702814
12. Oct 23:01:21, Step 1140, Loss = 0.705115
12. Oct 23:01:24, Step 1160, Loss = 0.698636
12. Oct 23:01:27, Step 1180, Loss = 0.697440
12. Oct 23:01:30, Step 1200, Loss = 0.698361
12. Oct 23:01:32, Step 1220, Loss = 0.696742
12. Oct 23:01:35, Step 1240, Loss = 0.698479


In [4]:
# always hanging around 0.693635
# "models/sl-v/tolf.ckpt" náði 0.55975 í fyrstu þjálfun!
# start =6e-5 virðist láta tolf-derivation lækka um 0.05 per 50 steps...
# Þegar ég hækka learning rate þá virðist allt ýtast niðður í 0.5f

In [5]:
def board_to_sanity(board, player):
    #TODO: Add a lot more heuristics!
    opponent = player * (-1)
    player_grid =  np.zeros((8,8))
    opponent_grid =  np.zeros((8,8))
    empties =  np.zeros((8,8))
    player_constant = np.zeros((8,8))
    zeros = np.zeros((8,8))
    legal_move_grid = np.zeros((8,8))
    ones = np.ones((8,8))
    corners = corner_board()
    x_squares = np.zeros((8,8))
    c_grid = c_squares()
    mobility = np.zeros((8,8))
    edges_a = edges_board_a()
    edges_b = edges_board_b()
    frontier = np.zeros((8,8))
    sum_player_stability = np.zeros((8,8))
    sum_opponent_stability = np.zeros((8,8))
    coin_parity = np.zeros((8,8))
    main_diagonal_1 = np.identity(8)
    main_diagonal_2 = np.fliplr(np.identity(8))
    
    corner_2x4_1 = corner_2x4()
    corner_2x4_2 = np.rot90(corner_2x4_1)
    corner_2x4_3 = np.rot90(corner_2x4_2)
    corner_2x4_4 = np.rot90(corner_2x4_3)
    corner_2x4_5 = np.transpose(corner_2x4_1)
    corner_2x4_6 = np.transpose(corner_2x4_2)
    corner_2x4_7 = np.transpose(corner_2x4_3)
    corner_2x4_8 = np.transpose(corner_2x4_4)
    
    
    #stability = np.zeros((8,8))
    for i in range(8):
        for j in range(8):
            #move_string = str((i+1)*10+j)
            #mobility[i][j] = opponent_mobility_after_move(board, move_string, player)
            if player == -1:
                player_constant[i][j] = 1
            if board[i][j] == player:
                player_grid[i][j] = 1
            elif board[i][j] != 0:
                opponent_grid[i][j] = 1
            else:
                empties[i][j] = 1

    for move in find_legal_moves(board, player):
        move = str(move)
        row = int(move[0]) - 1
        column = int(move[1]) - 1
        legal_move_grid[row][column] = 1
        mobility[row][column] = opponent_mobility_after_move(board, move, player)
        board_after_player_move = make_move(board, move, player)
        coin_parity[row, column] = np.sum(board_after_player_move == player) - np.sum(board_after_player_move == player*(-1))
        board_after_opponent_move = make_move(board, move, opponent)
        frontier[row, column] = calculate_frontier(board, (row, column))
        # Only calculate stability if any of the twelve indicated corner cells are occupied
        if board[0][0] != 0 or board[0][1] != 0 or board[1][0] != 0 or board[7][7] != 0 or board[6][7] != 0 or board[7][6] != 0 or board[0][7] != 0 or board[1][7] != 0 or board[0][6] != 0 or board[7][0] != 0 or board[7][1] != 0 or board[6][0] != 0:
            # Add feature plane that shows sum of stable friendly pieces
            # if current player places a move in legal move cell - and another
            # which shows sum of stable opponent pieces if opponent moves there
            ind_rows  = filled_rows(board_after_player_move)
            initial_rows = get_filled_row_features(ind_rows)
            ind_columns = filled_columns(board_after_player_move)
            ind_NW = filled_NW(board_after_player_move)
            ind_NE = filled_NE(board_after_player_move)
            initial_columns = get_filled_column_features(ind_columns)
            initial_NW = get_filled_NW_features(ind_NW)
            initial_NE = get_filled_NE_features(ind_NE)
            #TODO: Speed up stability calculations such that it utilizes it being monotonically increasing
            potential_player_stability = get_stability_features(board_after_player_move, player, initial_rows, initial_columns, initial_NW, initial_NE)
            potential_opponent_stability = get_stability_features(board_after_opponent_move, opponent, initial_rows, initial_columns, initial_NW, initial_NE)
            sum_player_stability[row][column] = np.sum(potential_player_stability)
            sum_opponent_stability[row][column] = np.sum(potential_opponent_stability)
        
    ind_rows  = filled_rows(board)    
    ind_columns = filled_columns(board)
    ind_NW = filled_NW(board)
    ind_NE = filled_NE(board)
    initial_rows = get_filled_row_features(ind_rows)
    initial_columns = get_filled_column_features(ind_columns)
    initial_NW = get_filled_NW_features(ind_NW)
    initial_NE = get_filled_NE_features(ind_NE)
    current_stability = get_stability_features(board, player, initial_rows, initial_columns, initial_NW, initial_NE)
    #TODO: Add these features again after speeding up the stability calculations
    #current_player_stability = get_stability_features(board, player, stability, rows, columns, NW, NE)
    #current_opponent_stability = get_stability_features(board, opponent, stability, rows, columns, NW, NE)
    x_squares[1][1] = 1
    x_squares[6][1] = 1
    x_squares[6][6] = 1
    x_squares[1][6] = 1
    
    

    return np.concatenate((mobility[:,:,None], sum_player_stability[:,:,None], sum_opponent_stability[:,:,None],
                          current_stability[:,:,None], frontier[:,:,None], coin_parity[:,:,None]), axis=2)

## np.random.rand(1)
